<a href="https://colab.research.google.com/github/griffinsnest/OneHotEncoding/blob/main/oneshotencoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

university_towns = []

with open('/content/university_towns.txt') as file:
  for line in file:
    if '[edit]' in line:
      state = line
    else:
      university_towns.append((state, line))

university_towns[:10]

towns_df = pd.DataFrame(university_towns, columns=['State', 'RegionName'])

def get_city(item):
  if " (" in item:
    return item[:item.find(')')+1]
  elif '[' in item:
    return item[:item.find('[')]
  else:
    return item

towns_df = towns_df.applymap(get_city)

new = towns_df["RegionName"].str.split(" \(", n = 1, expand = True)
  
# making separate first name column from new data frame
towns_df["City"]= new[0]
  
# making separate last name column from new data frame
towns_df["University"]= new[1]

towns_df["University"]= towns_df["University"].str.replace(')', '')
  
# Dropping old Name columns
towns_df.drop(columns =["RegionName"], inplace = True)

label_encoder = LabelEncoder()

towns_df['StateEncoded'] = label_encoder.fit_transform(towns_df['State'])

towns_df['ScldStateEncoded'] = round(towns_df['StateEncoded'].apply(lambda x: (x - towns_df['StateEncoded'].mean()) / (towns_df['StateEncoded'].std()) ), 2)

towns_df = towns_df.dropna()

states = towns_df['State']

towns_df = towns_df.drop(['State'], axis=1)

#print(towns_df.head())

data_column_category = towns_df.select_dtypes(exclude=[np.number]).columns
data_column_numeric = towns_df.select_dtypes(include=[np.number]).columns

print("\nWe don't want to onehot encode all the data as we still want to keep the target State data unique in order properly test with linear regression later")



df_oneshot_getdummies = pd.get_dummies(towns_df[data_column_category], prefix=data_column_category)


final_towns_df = pd.concat([df_oneshot_getdummies, towns_df[data_column_numeric]], axis=1)

final_towns_df = final_towns_df.dropna()

final_towns_df['State'] = states

features = final_towns_df[final_towns_df.columns.difference(['StateEncoded', 'ScldStateEncoded', 'State'])] #df_oneshot_getdummies 
print('\nThe features of the model used to find the target are the onehot encoded versions of:  ', data_column_category, sep="")
target = final_towns_df['ScldStateEncoded']
print('\nThe target of the model trying to be found with the feature is the scaled, label encoded versions of States called: ', final_towns_df.columns[-2], sep="")

X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=.25)

xb = X_train
yb = np.array(Y_train)
yb = yb.reshape(-1,1)

xb_test = X_test
yb_test = np.array(Y_test)
yb_test = yb_test.reshape(-1,1)
TestModel = lm.LinearRegression().fit(xb,yb)
TMscore = TestModel.score(xb_test, yb_test)

print('\nThe Test model of the scaled States has an linear regression accuracy of:  ', TMscore, sep="")
print("\nThe Test model of the scaled States having a linear regression in the negatives indicates we can't make accurate predictions of the States from the cities and Universities" ,sep="")
#print('\nThe parameters for the scaled test model are :\n m = ', TestModel.coef_, '\n b = ', TestModel.intercept_)

target2 = final_towns_df['StateEncoded']

print('\nThe new target of the model trying to be found with the feature is the label encoded versions States called: ', final_towns_df.columns[-3], sep="")

X_train, X_test, Y_train, Y_test = train_test_split(features, target2, test_size=.25)

xb = X_train
yb = np.array(Y_train)
yb = yb.reshape(-1,1)

xb_test = X_test
yb_test = np.array(Y_test)
yb_test = yb_test.reshape(-1,1)
TestModel = lm.LinearRegression().fit(xb,yb)
TMscore = TestModel.score(xb_test, yb_test)

print('\nThe Test model of the non-scaled States has an linear regression accuracy of:  ', TMscore, sep="")
print("\nThe Test model of the non-scaled States having a linear regression in the negatives indicates we can't make accurate predictions of the States from the cities and Universities" ,sep="")

print("\nAs you can see the linear regressions of both sources do not differ that much, both consistantly getting some number in the low negatives. The best they can get is about -.1 but even then they're still very close.")
print(' This is evidence to show that while there is slight variation between the two, there is not much difference between using the scaled target vs the non-scaled target.', sep="")


We don't want to onehot encode all the data as we still want to keep the target State data unique in order properly test with linear regression later

The features of the model used to find the target are the onehot encoded versions of:  Index(['City', 'University'], dtype='object')

The target of the model trying to be found with the feature is the scaled, label encoded versions of States called: ScldStateEncoded

The Test model of the scaled States has an linear regression accuracy of:  -0.028146677291102762

The Test model of the scaled States having a linear regression in the negatives indicates we can't make accurate predictions of the States from the cities and Universities

The new target of the model trying to be found with the feature is the label encoded versions States called: StateEncoded

The Test model of the non-scaled States has an linear regression accuracy of:  -0.1083071147130783

The Test model of the non-scaled States having a linear regression in the negatives in